# Data Cleaning:
Upon collecting the data and exporting into a .csv file we imported it back to begin the cleaning process

## Overview
The web scrapping and API pull brought us a lot of usefull information for our analysis; however, there were a lot of ways that we cleamed the data with before we could dive further into our analysis. Using primarily Pandas we utilized a various range of its tools from: 
* Adding/Dropping: Columns & Rows
* Lambda functions
* Splits, Strips, Remove
* Changing the type (ex: str -> int)
* Subsetting

### Import Packages

In [102]:
import csv
import pandas as pd
import numpy as np

### Import .csv into Pandas Data Frame

In [101]:
df_final = pd.read_csv('final_movie_data.csv')

In [106]:
# Quick search to overview what columns and types of data is in the data frame
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3361 entries, 0 to 3360
Data columns (total 32 columns):
Unnamed: 0               3361 non-null int64
Title                    3361 non-null object
Rank_by_MPAA_Rating      3361 non-null object
Domestic_Gross           3361 non-null object
Overall_Rank             3361 non-null object
Year                     3361 non-null object
Rating                   3361 non-null object
imdb_id                  3361 non-null object
adult                    3361 non-null bool
backdrop_path            3205 non-null object
belongs_to_collection    1145 non-null object
budget                   3361 non-null int64
genres                   3361 non-null object
homepage                 1382 non-null object
id                       3361 non-null int64
original_language        3361 non-null object
original_title           3361 non-null object
overview                 3360 non-null object
popularity               3361 non-null float64
poster_path            

In [108]:
# Show all the columns that have NaN values and sort high to low
df_final.isnull().sum().sort_values(ascending = False)

belongs_to_collection    2216
homepage                 1979
tagline                   251
backdrop_path             156
poster_path                13
overview                    1
vote_count                  0
adult                       0
budget                      0
Rating                      0
imdb_id                     0
Year                        0
Overall_Rank                0
Domestic_Gross              0
Rank_by_MPAA_Rating         0
Title                       0
genres                      0
original_language           0
id                          0
vote_average                0
original_title              0
popularity                  0
production_companies        0
production_countries        0
release_date                0
revenue                     0
runtime                     0
spoken_languages            0
status                      0
title                       0
video                       0
Unnamed: 0                  0
dtype: int64

### Removing Columns & Rows

There were a lot of columns that were deemed unnecessary or repeated so they were dropped in order to have only the essential data points needed.

For the rows after further investigation we realized that there were empty [ ] under the production columns that needed to be removed first before we could utilize a lambda function to clean the data.

In [109]:
# Drop columns that are not needed in data set. This also took care of most of the NaN values
df_final.drop(['homepage','tagline','backdrop_path','belongs_to_collection',
               'poster_path','Unnamed: 0','adult','id','original_title',
               'overview','spoken_languages','status','title','video',
              'Rank_by_MPAA_Rating','Overall_Rank'], axis = 1, inplace = True)

In [110]:
# Drop rows with [] as production company and production country
df_final.drop([142,151,153,166,167,210,211,222,248,252,262,264,270,272,273,277,279,280,282,289,303,309,
               310,312,317,320,324,327,337,339,341,347,348,350,354,356,357,358,850,954,1084,1111,1113,1179,
               1217,1225,1243,1248,1260,1268,1275,1307,2037,2140,2317,170,228,246,338,342,891,962,1060,1109,
               1192,1289,1311,2052,2264], inplace = True)

In [111]:
# Original data frame contained 3361 rows 32 columns
df_final.shape

(3292, 16)

### Cleaning Existing Columns & Creating New Columns

For genres, production_companies & production_countries the columns contained a lot of clutter. To tackle this we created new columns by taking the one main genre, company and country from each row. We utilized various functions such as lambda, strip, split, and replace in order to clean the data.

In [113]:
# Before clean
df_final['production_companies'][0]

"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfymY85KHT.png', 'name': 'Walt Disney Pictures', 'origin_country': 'US'}, {'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUHpPEwLf7.png', 'name': 'Pixar', 'origin_country': 'US'}]"

In [114]:
# Separate list of production companies to the main company by utilizing split
df_final['main_production'] = df_final['production_companies'].map(lambda x: x.split(':')[3])
df_final['main_production'] = df_final['main_production'].map(lambda x: x.split(',')[0])

In [118]:
# After cleaning shows Universal Pictures were responsible for creating the most movies
df_final['main_production'].value_counts(ascending = False)

 'Universal Pictures'             196
 'Paramount'                      181
 'Columbia Pictures'              132
 'Walt Disney Pictures'           129
 'New Line Cinema'                 76
                                 ... 
 'MICA Entertainment'               1
 'Lantica Media'                    1
 'Monkey Dance Productions'         1
 'Frank Yablans Presentations'      1
 'Tatiale Films'                    1
Name: main_production, Length: 1076, dtype: int64

In [116]:
# Separate list of production ccountries to one country by utilizing split and replace
df_final['main_production_country'] = df_final['production_countries'].map(lambda x: x.split(':')[2])
df_final['main_production_country'] = df_final['main_production_country'].map(lambda x: x.split('{')[0])
df_final['main_production_country'] = df_final['main_production_country'].map(lambda x: x.replace('}]',''))
df_final['main_production_country'] = df_final['main_production_country'].map(lambda x: x.replace('},',''))

In [120]:
# US companies are responsible for almost all of the worldwide blockbuster movies
df_final['main_production_country'].value_counts(ascending = False)

 'United States of America'     2420
 'United Kingdom'                188
 'Canada'                        109
 'Germany'                       100
 'United States of America'       85
 'France'                         68
 'Australia'                      50
 'United Kingdom'                 36
 'China'                          30
 'Japan'                          20
 'France'                         17
 'Czech Republic'                 13
 'New Zealand'                    12
 'Japan'                          12
 'Australia'                      11
 'Hong Kong'                      11
 'Ireland'                        11
 'Spain'                          10
 'Mexico'                          8
 'India'                           8
 'Italy'                           7
 'Belgium'                         6
 'Canada'                          6
 'United Arab Emirates'            4
 'China'                           3
 'South Korea'                     3
 'Sweden'                          3
 

In [122]:
# Separate list of genres for each movie into one main genre to categorize by utilizing split and replace
df_final['main_genre'] = df_final['genres'].map(lambda x: x.split(':')[2]).str.replace('}, ','')
df_final['main_genre'] = df_final['main_genre'].map(lambda x: x.split('{')[0])
df_final['main_genre'] = df_final['main_genre'].map(lambda x: x.replace('}]',''))

In [123]:
# Comedy is the most popular genre for blockbuster movies
df_final['main_genre'].value_counts(ascending = False)

 'Comedy'             798
 'Drama'              561
 'Action'             554
 'Adventure'          350
 'Animation'          165
 'Horror'             155
 'Crime'              121
 'Fantasy'            117
 'Thriller'           113
 'Science Fiction'     75
 'Family'              69
 'Romance'             65
 'Documentary'         58
 'Mystery'             31
 'Music'               21
 'Western'             15
 'History'             13
 'War'                 11
Name: main_genre, dtype: int64

### Formatting Financial Numbers

The financial numbers from the Box Office Mojo web scrapping came in as strings with symbols ($,). In order to compare and manipulate all the financial numbers together we reformatted these columns to all be integers without any symbols.

Additionally, we were able to calculate **foreign investments** by subtracting **domestic revenue** from Box Office Mojo from **total revenue** from TMDB

In [124]:
# Matched Domestic_Gross by removing $ and , also changing to an int
df_final['Domestic_Gross'] = df_final['Domestic_Gross'].map(lambda x: x.strip('$'))
df_final['Domestic_Gross'] = df_final['Domestic_Gross'].map(lambda x: x.replace(',',''))
df_final['Domestic_Gross'] = df_final['Domestic_Gross'].astype(int)

In [125]:
# Calculated Foreign_Gross revenue numbers by subtracting Domestic from the total revenue
df_final['Foreign_Gross'] = df_final['revenue'] - df_final['Domestic_Gross']

In [126]:
# Calculated Profit by subtracting revenue by budget
df_final['Profit'] = df_final['revenue'] - df_final['budget']

In [127]:
df_final.head()

,Title,Domestic_Gross,Year,Rating,imdb_id,budget,genres,original_language,popularity,production_companies,...,release_date,revenue,runtime,vote_average,vote_count,main_production,main_production_country,main_genre,Foreign_Gross,Profit
0,Toy Story 4,434038008,2019,G,tt1979376,175000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",en,34.489,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",...,2019-06-19,1073394593,100,7.6,5029,'Walt Disney Pictures','United States of America','Adventure',639356585,898394593
1,The Lion King,422783777,1994,G,tt0110357,45000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",en,32.284,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",...,1994-05-07,987483777,89,8.3,12393,'Walt Disney Pictures','United States of America','Family',564700000,942483777
2,Toy Story 3,415004880,2010,G,tt0435761,200000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",en,27.214,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...",...,2010-06-16,1066969703,103,7.8,10168,'Pixar','United States of America','Animation',651964823,866969703
3,Finding Nemo,380843261,2003,G,tt0266543,94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",en,34.417,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...",...,2003-05-30,940335536,100,7.8,13547,'Pixar','United States of America','Animation',559492275,846335536
4,"Monsters, Inc.",289916256,2001,G,tt0198781,115000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,41.313,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...",...,2001-11-01,562816256,92,7.8,12668,'Pixar','United States of America','Animation',272900000,447816256


### Subsetting

We quickly realized that not all of the information coming from TMDB were accurate. In TMDB there were movies that were listed as 0 in revenue or just a few $. On the flip side, Box Office Mojo has reliable financial numbers. In order to prevent these outliers from skewing our data we removed any movies where there were 0 in revenue or large discrepencies between the two data sources.

In [132]:
# We created a new data frame that would filter out any rows where the revenue numbers were incorrect
df = df_final[df_final['revenue'] >= df_final['Domestic_Gross']]
df.head()

,Title,Domestic_Gross,Year,Rating,imdb_id,budget,genres,original_language,popularity,production_companies,...,release_date,revenue,runtime,vote_average,vote_count,main_production,main_production_country,main_genre,Foreign_Gross,Profit
0,Toy Story 4,434038008,2019,G,tt1979376,175000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",en,34.489,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",...,2019-06-19,1073394593,100,7.6,5029,'Walt Disney Pictures','United States of America','Adventure',639356585,898394593
1,The Lion King,422783777,1994,G,tt0110357,45000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",en,32.284,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",...,1994-05-07,987483777,89,8.3,12393,'Walt Disney Pictures','United States of America','Family',564700000,942483777
2,Toy Story 3,415004880,2010,G,tt0435761,200000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",en,27.214,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...",...,2010-06-16,1066969703,103,7.8,10168,'Pixar','United States of America','Animation',651964823,866969703
3,Finding Nemo,380843261,2003,G,tt0266543,94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",en,34.417,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...",...,2003-05-30,940335536,100,7.8,13547,'Pixar','United States of America','Animation',559492275,846335536
4,"Monsters, Inc.",289916256,2001,G,tt0198781,115000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,41.313,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...",...,2001-11-01,562816256,92,7.8,12668,'Pixar','United States of America','Animation',272900000,447816256


In [133]:
df.shape

(2728, 21)

### Export CSV File

In [134]:
# Export the new cleaned data frame into a csv to be able to use for data visualizations
import csv
df.to_csv('cleaned_final_movie_data.csv')